**Work in Progress**

### Reference : https://www.analyticsvidhya.com/blog/2018/07/hands-on-sentiment-analysis-dataset-python/

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
df=pd.read_csv('../input/pfizer-vaccine-tweets/vaccination_tweets.csv')

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

# Basic EDA

In [ ]:
sns.countplot(data=df,x='user_verified').set_title('Verified Users')

In [ ]:
df['timestamp'] = df['date'].apply(lambda x : pd.to_datetime(str(x)))
df['Month']=df['timestamp'].dt.month
df['date']=df['timestamp'].dt.date

In [ ]:
sns.countplot(data=df,x='Month').set_title('No.of Tweets by Month')

In [ ]:
df.sort_values('date')

plt.figure(figsize=(15,8))
ax=sns.countplot(data=df,x='date').set_title('No.of Tweets by Date')
locs, labels = plt.xticks()
plt.setp(labels, rotation=60)
ax

In [ ]:
fig = px.histogram(df, x=df['date'],title='No.of Tweets by Date')
fig.update_traces(marker_color='green')
fig

In [ ]:
fig = px.histogram(df, x=df['text'].str.len(),nbins=15,title='Tweets Length')
fig

# Natural Language Processing

In [ ]:
import string
import nltk
import warnings
import re
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# Removing Twitter Handles
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [ ]:
df['text_new'] = np.vectorize(remove_pattern)(df['text'], "@[\w]*")

In [ ]:
# Removing Punctuations, Numbers, and Special Characters
df['text_new'] = df['text_new'].str.replace("[^a-zA-Z#]", " ")

In [ ]:
# Removing Short Words
df['text_new'] = df['text_new'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [ ]:
df['text_new']

In [ ]:
# Tokenization
tokenized_tweet = df['text_new'].apply(lambda x: x.split())
tokenized_tweet.head()

In [ ]:
# Stemming
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet.head()

In [ ]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
df['text_new'] = tokenized_tweet

## Wordcloud

In [ ]:
df['text_new']

In [ ]:
all_words = ' '.join([text for text in df['text_new']])
from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()